In [15]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime
import requests
from config import weather_api_key

In [16]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=02936facfbe1b47412c990bf5012c74e


In [17]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [18]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [19]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

618

In [20]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")    

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | busselton
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | arraial do cabo
Processing Record 4 of Set 1 | vardo
Processing Record 5 of Set 1 | hermanus
Processing Record 6 of Set 1 | kodiak
Processing Record 7 of Set 1 | hobart
Processing Record 8 of Set 1 | la ronge
Processing Record 9 of Set 1 | severodvinsk
Processing Record 10 of Set 1 | barentsburg
City not found. Skipping...
Processing Record 11 of Set 1 | hilo
Processing Record 12 of Set 1 | ponta delgada
Processing Record 13 of Set 1 | vanimo
Processing Record 14 of Set 1 | castro
Processing Record 15 of Set 1 | bredasdorp
Processing Record 16 of Set 1 | fare
Processing Record 17 of Set 1 | ushuaia
Processing Record 18 of Set 1 | makaha
Processing Record 19 of Set 1 | yellowknife
Processing Record 20 of Set 1 | thompson
Processing Record 21 of Set 1 | kahului
Processing Record 22 of Set 1 | owando
Processing Record 23

In [21]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,52.09,84,86,11.97,AU,2021-07-19 18:14:24
1,Rikitea,-23.1203,-134.9692,70.38,65,98,25.01,PF,2021-07-19 18:19:17
2,Arraial Do Cabo,-22.9661,-42.0278,69.76,64,75,17.27,BR,2021-07-19 18:19:17
3,Vardo,70.3705,31.1107,48.40,87,75,13.80,NO,2021-07-19 18:15:10
4,Hermanus,-34.4187,19.2345,51.91,82,99,7.58,ZA,2021-07-19 18:15:46
5,Kodiak,57.7900,-152.4072,60.31,77,1,6.91,US,2021-07-19 18:19:17
6,Hobart,-42.8794,147.3294,49.26,74,99,3.00,AU,2021-07-19 18:15:04
7,La Ronge,55.1001,-105.2842,61.05,63,75,8.05,CA,2021-07-19 18:19:18
8,Severodvinsk,64.5635,39.8302,60.91,72,25,1.68,RU,2021-07-19 18:19:18
9,Hilo,19.7297,-155.0900,79.29,94,90,6.91,US,2021-07-19 18:14:38


In [22]:
new_columns = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']

city_data_df = city_data_df[new_columns]

city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Busselton,AU,2021-07-19 18:14:24,-33.6500,115.3333,52.09,84,86,11.97
1,Rikitea,PF,2021-07-19 18:19:17,-23.1203,-134.9692,70.38,65,98,25.01
2,Arraial Do Cabo,BR,2021-07-19 18:19:17,-22.9661,-42.0278,69.76,64,75,17.27
3,Vardo,NO,2021-07-19 18:15:10,70.3705,31.1107,48.40,87,75,13.80
4,Hermanus,ZA,2021-07-19 18:15:46,-34.4187,19.2345,51.91,82,99,7.58
5,Kodiak,US,2021-07-19 18:19:17,57.7900,-152.4072,60.31,77,1,6.91
6,Hobart,AU,2021-07-19 18:15:04,-42.8794,147.3294,49.26,74,99,3.00
7,La Ronge,CA,2021-07-19 18:19:18,55.1001,-105.2842,61.05,63,75,8.05
8,Severodvinsk,RU,2021-07-19 18:19:18,64.5635,39.8302,60.91,72,25,1.68
9,Hilo,US,2021-07-19 18:14:38,19.7297,-155.0900,79.29,94,90,6.91


In [24]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")